In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
%%time
import pandas as pd
import re
data = pd.read_excel("/content/flipkart_data.xlsx")


def madhur(source):
    filtered_data=data[(data['componentClassName']== source) & (data['appName']==source)]
    filtered_data.Text.to_list()
    concatenated_string=" ".join(str(i) for i in filtered_data['Text'])
    #from langchain.chat_models import AzureChatOpenAI
    #from langchain.chat_models import ChatOpenAI
    # from langchain.chains import GraphCypherQAChain
    # from langchain.graphs import Neo4jGraph
    # from langchain.schema import HumanMessage
    # llm = AzureChatOpenAI(
    #         openai_api_base='https://xgptopenai.openai.azure.com/',
    #         openai_api_version="2023-05-15",
    #         deployment_name='xgptopenai',
    #         openai_api_key='174d1e2747d54e5291fa10b7f6cb8286',
    #         openai_api_type="azure",
    #         temperature=0
    #     )
    cleaned_df = filtered_data['Text'].drop_duplicates(keep='last')
    clean_data=" ".join(str(i) for i in cleaned_df)
    """Products for chekout"""
    import re
    match = re.search(r'\ Place order\b', clean_data)
    pattern = re.compile(r'\ Place order\b')

    # Find all matches in the passage
    matches = pattern.finditer(clean_data)

    # Get the positions of all matches
    positions = [match.span() for match in matches]

    place_order_context=clean_data[:positions[0][1]]
    "Adding qunatity pattern to pace order clean data context"
    """Products for chekout and quatity keyword"""
    match = re.search(r'\ Quantity\b', place_order_context)
    pattern = re.compile(r'\ Quantity\b')

    # Find all matches in the passage
    matches = pattern.finditer(place_order_context)

    # Get the positions of all matches
    positions_quantity = [match.span() for match in matches]

    fl_qunatiy=[]
    start_pos=0
    for i in positions_quantity:
        temp_list=[]
        end_pos=list(i)[1]
        temp_list.append(start_pos)
        temp_list.append(end_pos)
        fl_qunatiy.append(temp_list)
        start_pos=end_pos

    products_bought=[]
    for i in positions_quantity:
        products_bought.append(place_order_context[:i[1]][::-1][:400][::-1])

        #%%time
    """Rephrasing the products description"""
    rephrased_context_products=[]
    for i in products_bought:
        r = model.generate_content("Rephrase the following"+str(i))
        rephrased_context_products.append(r.text)

    def product_details_add_to_cart(string_injection_index):
        """Extract information of product based on the index of Add to cart indexes"""

        #Prompt Enginerring to Answer Questions
        """Product Sub-Category"""
        message_subcategory = model.generate_content("What is the sub-category of the product.Just extract the sub-category from the context."+string_injection_index)
        output_subcategory= message_subcategory


        #Prompt Enginerring to Answer Questions
        """Product Category"""
        message_category = model.generate_content("What is the product category.Just name the category?"+str(output_subcategory.text))
        output_category= message_category


        #Prompt Enginerring to Answer Questions
        """Product Colour"""
        message_colour= model.generate_content ("What is the product colour.Just tell the name of the colour.No need to write complete statement?"+string_injection_index)
        output_colour= message_colour



        #Prompt Enginerring to Answer Questions
        """Product Price"""
        message_price= model.generate_content ("What is the price of the product added to cart.Extract only the numbers."+string_injection_index)
        output_price= message_price
        message_price_1 = model.generate_content ("Extract only the numbers."+str(output_price.text))
        output_price= message_price_1



        #Prompt Enginerring to Answer Questions
        """Product Seller"""
        message_seller= model.generate_content ("Where is the  product that is added to the cart coming from."+string_injection_index)
        output_seller= message_seller
        message_seller_1= model.generate_content ("Extract the product rating."+str(output_seller.text))
        output_seller_1= message_seller_1


        #Prompt Enginerring to Answer Questions
        """Product Specifications"""
        message_specifications = model.generate_content ("What are the product technical specifications for the product that is added to cart?"+string_injection_index)
        output_specifications= message_specifications
        #Prompt Enginerring to Answer Questions
        """Product Specifications"""
        message_specifications= model.generate_content ("Remove the price,seller and the colour information and extract the remaining information fo the product."+str(output_specifications.text))
        output_specifications_1= message_specifications
        structured_message= model.generate_content (
            "What is the rating of the product according to the context?Extract only the rating no."+str(string_injection_index))
        output_rating= structured_message
        structured_message= model.generate_content (
            "Extract only the rating no. from the context"+str(output_rating.text))
        output_rating= structured_message

        return output_category,output_subcategory,output_colour,output_price,output_seller_1,output_specifications_1,output_rating


    #%%time
    final_output=[]
    for all_selected_products in rephrased_context_products:
        output_category,output_subcategory,output_colour,output_price,output_seller_1,output_specifications_1,output_rating=product_details_add_to_cart(all_selected_products)
        final_output.extend([output_category,output_subcategory,output_colour,output_price,output_seller_1,output_specifications_1,output_rating])

    final_output_messages=[]
    start_index=0
    for i in range(int(len(final_output)/7)):
        final_index=start_index+7
        final_output_messages.append((final_output)[start_index:final_index])
        start_index=final_index

    import re
    import pandas as pd

    # Sample statement
    additional_information_revised = []
    for i in range(len(final_output_messages)):
        statement = final_output_messages[i][5].text

        # Regular expression pattern
        pattern = r"Product\s\d+\s*:"
        matches = re.finditer(pattern, statement)
        matches_no = re.findall(pattern, statement)

        # Get the index position of the last match
        last_match_index = None
        for match in matches:
            last_match_index = match.end()

        # Print the index position of the last match
        #print("Index position of the last match:", last_match_index)
        if len(matches_no) > 1:
            additional_information_revised.append(final_output_messages[i][5].text[last_match_index:])
        else:
            additional_information_revised.append(final_output_messages[i][5].text)

    category_final = []
    sub_category_final = []
    colour_final = []
    price_final = []
    output_seller_final = []
    additional_info = []
    output_rating = []

    for i in range(len(final_output_messages)):
        category_final.append(final_output_messages[i][0].text)
        sub_category_final.append(final_output_messages[i][1].text)
        colour_final.append(final_output_messages[i][2].text)
        price_final.append(final_output_messages[i][3].text)
        output_seller_final.append(final_output_messages[i][4].text)
        additional_info.append(final_output_messages[i][5].text)
        output_rating.append(final_output_messages[i][6].text)

    df_output = pd.DataFrame()
    df_output['category_final'] = category_final
    df_output['sub_category_final'] = sub_category_final
    df_output['colour_final'] = colour_final
    df_output['price_final'] = price_final
    df_output['output_seller_final'] = output_seller_final
    df_output['additional_info'] = additional_info
    df_output['output_rating'] = output_rating

    # maggi chicken cubes,Bakson's Homoeopathy AC,6 bars of Nestle Munch Chocolate Crunchy Wa
    rows_to_drop = []
    for i in range(len(df_output)):
        if 'Unfortunately' in df_output['additional_info'][i]:
            rows_to_drop.append(i)
        if len(df_output['price_final'][i].split(",")) > 1:
            rows_to_drop.append(i)

    # Drop the rows after iterating
    df_output = df_output.drop(rows_to_drop)

    # Reset the index to avoid gaps after dropping rows
    df_output = df_output.reset_index(drop=True)

    return df_output